## Simulate sequence and phenotype data
This notebook is for development purposes only.

#### TODO
* Simulate patient phenotype data
* \[Optional\] generate sequences without having to worry about duplicates (as done with the variants)

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)
aa = "ACDEFGHIKLMNPQRSTVWY"

aa_to_index = {k:i for i,k in enumerate(aa)}
index_to_aa = {i:k for i,k in enumerate(aa)}

# Troponin C
refseq = "MDDIYKAAVEQLTEEQKNEFKAAFDIFVLGAEDGCISTKELGKVMRMLGQNPTPEELQEMIDEVDEDGSGTVDFDEFLVMMVRCMKDDSKGKSEEELSDLFRMFDKNADGYIDLDELKIMLQATGETITEDDIEELMKDGDKNNDGRIDYDEFLEFMKGVE"
ref_onehot = np.identity(20)[[aa_to_index[x] for x in refseq],:]

### Generate variants table

In [2]:
# brute force (list all possible variants and sample) to avoid replicates
variants = []
for pos in range(len(refseq)):
    for alt_ in np.arange(1,20):
        alt = (aa_to_index[refseq[pos]]+alt_)%20
        variants.append((pos, 
                         aa_to_index[refseq[pos]], 
                         alt,
                         refseq[pos]+str(pos)+index_to_aa[alt]))
        
variants_idxs = np.random.choice(len(variants), size = 150, replace=False)
variants = [variants[i] for i in variants_idxs]

var_table = pd.DataFrame({'pos': [v[0] for v in variants], 
                          'ref': [v[1] for v in variants], 
                          'alt': [v[2] for v in variants], 
                          'id': [v[3] for v in variants]})

assert len(var_table.drop_duplicates(subset='id')) == 150

In [8]:
var_table

,pos,ref,alt,id
0,92,15,12,S92P
1,10,13,7,Q10I
2,71,17,5,V71G
3,76,4,12,F76P
4,83,1,2,C83D
...,...,...,...,...
145,91,8,1,K91C
146,141,8,19,K141Y
147,58,3,5,E58G
148,71,17,19,V71Y


### Generate sequences table

In [3]:
seq_nvars = np.random.poisson(1, size=200)+1

seq_varpos = [np.random.choice(list(set(var_table['pos'])), 
                               size = n, replace=False) for n in seq_nvars]

seq_varids = []
for varpos in seq_varpos:
    varids = []
    for pos in varpos:
        varids.append(np.random.choice(var_table[var_table['pos']==pos]['id']))
    seq_varids.append('_'.join(sorted(varids)))

seq_table = pd.DataFrame({'n_vars':[0], 'v_ids':['']})
seq_table = pd.concat((seq_table, pd.DataFrame({'n_vars':seq_nvars, 'v_ids':seq_varids})))
seq_table = seq_table.drop_duplicates(subset='v_ids', ignore_index=True)
seq_table.index.name = 'seq_id'

In [4]:
seq_table

,n_vars,v_ids
seq_id,,
0,0,
1,1,M46F
2,1,G33T
3,1,C83D
4,1,S97Y
...,...,...
182,2,E160R_I111H
183,1,M119L
184,3,F73A_G69K_L153C


### Generate sequence data

In [35]:
seq_data = np.repeat(ref_onehot[np.newaxis, :, :], 
                     repeats = len(seq_table)+1, 
                     axis =0)

for seq_id, vids in enumerate(seq_table['v_ids']):
    if seq_id>0:
        for vid in vids.split('_'):
            ref, pos, alt = aa_to_index[vid[0]], \
                            int(vid[1:-1]), \
                            aa_to_index[vid[-1]]
            assert seq_data[seq_id, pos, ref] == 1
            seq_data[seq_id, pos, :] = 0
            seq_data[seq_id, pos, alt] = 1


# print(mut_seqs[:10])


In [42]:
mut_seqs = np.repeat(np.array(list(refseq))[np.newaxis,:],repeats=len(seq_table)+1,axis=0)

for seq_id, vids in enumerate(seq_table['v_ids']):
    if seq_id>0:
        for vid in vids.split('_'):
            ref, pos, alt = vid[0], int(vid[1:-1]), vid[-1]
            assert mut_seqs[seq_id, pos] == ref
            mut_seqs[seq_id,pos] = alt

mut_seqs = np.array([''.join(x) for x in mut_seqs])

In [43]:
mut_seqs

array(['...'],
      dtype='<U161')

### Save tables and matrix data

In [46]:
var_table.to_parquet("data/toy_data/variants.parquet")
seq_table.to_parquet("data/toy_data/sequences.parquet")
np.save("data/toy_data/seq_data.npy", seq_data)
np.save("data/toy_data/mutseqs.npy",mut_seqs)